In [1]:
import os
from shutil import copyfile
import matplotlib.pyplot as plt
import numpy as np
import cv2
import imageio
import scipy.misc
import glob
import pandas as pd

In [2]:
def rgb2L(img):
    maskAgent = cv2.inRange(img, (5,0,65), (15,2,75))
    maskLD = cv2.inRange(img, (60,62,22), (70,67,30))
    maskGD = cv2.inRange(img, (10,45,10), (50,105,50))
    maskKD = cv2.inRange(img, (60,30,30), (70,35,35))
    maskED = cv2.inRange(img, (27,30,25), (32,35,45))
    maskPD = cv2.inRange(img, (55,30,70), (70,45,80))
    maskK = cv2.inRange(img, (50,60,0), (70,80,20))
    maskO = cv2.inRange(img, (5,60,65), (15,74,85))
    maskW = cv2.inRange(img, (0,0,0), (2,2,2))
    maskF = cv2.inRange(img, (45,38,22), (60,50,26))
    maskP = cv2.inRange(img, (25,2,60), (40,8,70))
    
    sem = np.zeros((168,168))
    sem[maskAgent==255] = 1
    sem[maskLD==255] = 2
    sem[maskGD==255] = 3
    sem[maskKD==255] = 4
    sem[maskED==255] = 5
    sem[maskPD==255] = 6
    sem[maskK==255] = 7
    sem[maskO==255] = 8
    sem[maskW==255] = 9
    sem[maskF==255] = 10
    sem[maskP==255] = 11
    return sem

label_dict = {"Unknown": 0,
              "Agent": 1,
          "Level Door": 2,
          "Regular Door": 3 ,
          "Key Door": 4 ,
          "Entry Door": 5 ,
          "Puzzle Door": 6 ,
          "Key": 7 ,
          "Time Orb": 8 ,
          "Wall":9,
          "Floor": 10,
        "Puzzle":11}
inv_map = {v: k for k, v in label_dict.items()}

In [3]:
def getRewardPer(semantic):#XX merge this into one function
    #Returns percentage of pixels containing a rewarding object -> door, key, orb, exit, doorZone, lockedDoorRegular
    rewarding = [2,3,4,7,8]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getDoorPer(semantic):
    #Returns percentage of pixels containing a rewarding object -> door, key, orb, exit, doorZone, lockedDoorRegular
    rewarding = [2,3,4]#+8 for door area
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getKeyPer(semantic):
    #Returns percentage of pixels containing a rewarding object -> door, key, orb, exit, doorZone, lockedDoorRegular
    rewarding = [7]#+8 for door area
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getOrbPer(semantic):
    #Returns percentage of pixels containing a rewarding object -> door, key, orb, exit, doorZone, lockedDoorRegular
    rewarding = [8]#+8 for door area
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getLevelDoorPer(semantic):
    rewarding = [2]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getGreenDoorPer(semantic):
    rewarding = [3]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getKeyDoorPer(semantic):
    rewarding = [4]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getOtherDoorPer(semantic):
    rewarding = [5,6]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getPuzzlePer(semantic):
    rewarding = [11]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getFloorPer(semantic):
    rewarding = [10]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]
def getUnknownPer(semantic):
    rewarding = [0]
    r = 0
    for i in range(semantic.shape[0]):
        if semantic[i] in rewarding:
            r = r + 1
    return r/semantic.shape[0]

In [4]:
upper = 0.8
lower = 0.0001
def getClassification(img):
    l = rgb2L(img).flatten()
    classification = [0,0,0,0]
    lDoor = getLevelDoorPer(l)
    gDoor = getGreenDoorPer(l)
    kDoor = getKeyDoorPer(l)
    oDoor = getOtherDoorPer(l)
    maxDoor = np.max([lDoor,gDoor,kDoor])
    if getDoorPer(l)>upper or getFloorPer(l)<0.1 or getUnknownPer(l)>0.1:
        return [-1,-1,-1,-1]   
    if (lDoor>lower)&(lDoor==maxDoor):
        classification[0] = 1
    elif (gDoor>lower)&(gDoor==maxDoor):
        classification[0] = 2
    elif (kDoor>lower)&(kDoor==maxDoor):
        classification[0] = 3
    elif (oDoor>lower)&(classification[0]==0):#only this if no other door in view to prevent artifacts
        classification[0] = 4
        
    if getKeyPer(l)>lower:
        classification[1] = 1
    
    if getOrbPer(l)>lower:
        classification[2] = 1
    
    if getPuzzlePer(l)>lower*0.01:
        classification[3] = 1
    return classification


In [10]:
#source_path = './Results/TowerTraining/Recordings/CuriousNoR/2800_0/'# Careful here! folder already hand checked!
#dest_path = './Results/TowerTraining/Recordings/CuriousNoR/2800_0/Sorted/'

In [6]:
for i in range(0,2800):
    im_path = source_path + "Labels/img"+str(i)+".png"
    labels = imageio.imread(im_path)
    c = getClassification(labels)
    #print(str(i)+': '+str(c))
    try:
        copyfile(source_path+'/RGB/img'+str(i)+'.png', dest_path+str(c)+'/img'+str(i)+'.png')
    except:
        os.mkdir(dest_path+str(c))
        copyfile(source_path+'/RGB/img'+str(i)+'.png', dest_path+str(c)+'/img'+str(i)+'.png')

In [3]:
dest_path = './Results/TowerTraining/BalancedTestSetAll/'

In [4]:
labels = pd.DataFrame(columns=['ID','Door','Key','Orb','Puzzle'])
for folder in os.listdir(dest_path):
    folderLabel = folder[1:-1].split(',')
    for frame in os.listdir(dest_path + '/' + folder):
        frameNum = int(frame[3:].split('.')[0])
        labels = labels.append({'ID':frameNum,'Door':int(folderLabel[0]),'Key':int(folderLabel[1]),
                       'Orb':int(folderLabel[2]),'Puzzle':int(folderLabel[3])},ignore_index=True)
        
labels = labels.sort_values(['ID']).reset_index()[['ID','Door','Key','Orb','Puzzle']]
labels.to_csv(dest_path + 'labels.csv')